In [8]:
# base
import pandas as pd
import numpy as np
import os, time
from multiprocessing import Pool, cpu_count

# my function
from mosaic_rasters_functions import summarize_structure

In [2]:
# generate list of csv files to iterate through
csv_dir = os.path.join("..", "data", "all_vars", "sampled")
csv_list = []
for file in os.listdir(csv_dir):
    if file.endswith(".csv"):
        csv_list.append(os.path.join(csv_dir, file))

In [10]:
start_time = time.time()
if __name__ == '__main__':
    with Pool() as pool:
        out_dfs = pool.map(summarize_structure, csv_list)
        
pd.concat(out_dfs).to_csv(os.path.join("..", "data", "structure_means.csv"))
print(time.time() - start_time)

59.80458617210388


In [ ]:
def summarize_structure(file):
    print(file)

    loaded = pd.read_csv(file)
    csv = loaded[["protected", "subzone", "vlce", "elev_cv", "loreys_height", "percentage_first_returns_above_2m", "total_biomass"]]
    
    # removal of harvested and non-forest pixels is now done in the sampling step (R)
    #csv = csv[csv["vlce"].isin([230, 220, 210, 81])] # use only forested pixels
    #csv = csv[csv["Change_Attribution"] != 2] # remove harvested pixels
    csv = csv[csv["elev_cv"] < 1]
    
    output = csv.groupby(["protected", "subzone"]).agg(
        mean_elev_cv = ("elev_cv", "mean"),
        mean_loreys_height = ("loreys_height", "mean"),
        mean_percentage_first_returns_above_2m = ("percentage_first_returns_above_2m", "mean"),
        mean_total_biomass = ("total_biomass", "mean"),
        sd_elev_cv = ("elev_cv", np.std),
        sd_loreys_height = ("loreys_height", np.std),
        sd_percentage_first_returns_above_2m = ("percentage_first_returns_above_2m", np.std),
        sd_total_biomass = ("total_biomass", np.std),
        count = ("protected", "count"))
    
    return output

In [9]:
file = csv_list[0]
loaded = pd.read_csv(file)
csv = loaded[["protected", "subzone", "vlce", "elev_cv", "loreys_height", "percentage_first_returns_above_2m", "total_biomass"]]
np.percentile(csv["loreys_height"], 2.5)
np.percentile(csv)

6.048